Shape optimization
==================

## Setup

As always, let's import `drjit` and `mitsuba` and set a differentiation-aware variant.

In [ ]:
import matplotlib.pyplot as plt 
import os
import drjit as dr
import mitsuba as mi
import torch

mi.set_variant('cuda_ad_rgb', 'llvm_ad_rgb')
torch.cuda.set_device(0)
mi.util.dr.set_device(0)

## Batched rendering

In [ ]:
from mitsuba import ScalarTransform4f as T

scene_dict = {
    'type': 'scene',
    'integrator': {
        'type': 'direct_reparam',
    },
    'sensor': {
        'type': 'perspective',
        'fov': 4.8,
        'to_world': T.look_at(target=[0, 0, 1], origin=[0,0,0], up=[0, -1, 0]),
        'film': {
            'type': 'hdrfilm',
            'width': 256,
            'height': 256,
            'filter': {'type': 'gaussian'},
            'sample_border': True
        }
    },
    'emitter': {
        'type': 'envmap',
        'filename': "./output/redner-bikerman/envMap_0.png",
    },
    'shape': {
        'type': 'obj',
        'filename': "./output/Bikerman.jpg/debug/mesh/debug1_iter100.obj", # already optimised from step 1, already in camera verts
        'bsdf': {'type': 'diffuse'}
    }
}

scene_target = mi.load_dict(scene_dict)
# generate pictures from all view points
def plot_batch_output(out: mi.TensorXf):
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(mi.util.convert_to_bitmap(out))
    ax.axis('off');

In [ ]:
ref_img = mi.render(scene_target, spp=256)
plot_batch_output(ref_img)

In [ ]:
#get ref image from file
from image import Image, ImageFolder, overlayImage, saveImage
inputImage = Image('./input/detailled_faces_unsplash/BikerManCropped.jpg', 'cuda', 256)


In [ ]:
scene_source = mi.load_dict(scene_dict)

img = mi.render(scene_source, spp=256)
plot_batch_output(img)
# input tensor pretty much
input_img = inputTensor = torch.pow(inputImage.tensor, inputImage.gamma)

## Large Steps Optimization

In [ ]:
# !pip install cholespy
lambda_ = 25
params = mi.traverse(scene_source)

ls = mi.ad.LargeSteps(params['shape.vertex_positions'], params['shape.faces'], lambda_)
# We also use a slighlty modified version of the adam optimizer, that uses a uniform second moment for all parameters. This can be done by setting `uniform=True` when instantiating the optimizer.
opt = mi.ad.Adam(lr=1e-1, uniform=True)
# convert back and forth cartesian -> diff representation vertex coords.
# init latent variable
opt['u'] = ls.to_differential(params['shape.vertex_positions'])

#otpimisation 
iterations = 500 if 'PYTEST_CURRENT_TEST' not in os.environ else 5
for it in range(iterations):
    loss = mi.Float(0.0)

    # Retrieve the vertex positions from the latent variable
    params['shape.vertex_positions'] = ls.from_differential(opt['u'])
    params.update()

    current_img = mi.render(scene_source, params, seed=it, spp=32)

    # L1 Loss
    loss = dr.mean(dr.abs(current_img - mi.TensorXf(input_img.squeeze(0))))
    dr.backward(loss)
    opt.step()

    print(f"Iteration {1+it:03d}: Loss = {loss[0]:6f}", end='\r')

## Intermediate result

In [ ]:
# Update the mesh after the last iteration's gradient step
params['shape.vertex_positions'] = ls.from_differential(opt['u'])
params.update();

final_img = mi.render(scene_source, spp=128)
plot_batch_output(ref_img) # start
plot_batch_output(mi.TensorXf(input_img.squeeze(0))) # input
plot_batch_output(final_img) # final

## Remeshing

The result above can be further improved with the help of remeshing. By increasing the tesselation of the mesh, we will be able to recover more details of the target shape. Intuitively, the intent of this step is similar to other "coarse-to-fine" optimization strategies. For example, in the [caustics][1] or the [volume optimization][2] tutorial we increase the resolution of texture that is being optimized over time.

We will use the Botsch-Kobbelt remeshing algorithm provided by the `gpytoolbox` package:

[1]: https://mitsuba.readthedocs.io/en/stable/src/inverse_rendering/caustics_optimization.html
[2]: https://mitsuba.readthedocs.io/en/stable/src/inverse_rendering/volume_optimization.html

In [ ]:
# !pip install gpytoolbox
import numpy as np
import torch 

torch.cuda.empty_cache()

v_np = params['shape.vertex_positions'].numpy().reshape((-1,3)).astype(np.float64)
f_np = params['shape.faces'].numpy().reshape((-1,3))
# The Botsch-Kobbelt remeshing algorithm takes a "target edge length" as input argument. This controls the desired tesselation of the mesh. 
# Since we want to increase resolution, we will set this as half of the mean edge length of the current mesh.
# Compute average edge length
l0 = np.linalg.norm(v_np[f_np[:,0]] - v_np[f_np[:,1]], axis=1)
l1 = np.linalg.norm(v_np[f_np[:,1]] - v_np[f_np[:,2]], axis=1)
l2 = np.linalg.norm(v_np[f_np[:,2]] - v_np[f_np[:,0]], axis=1)

target_l = np.mean([l0, l1, l2]) / 2

#We can now run the Botsch-Kobbelt remeshing algorithm. It runs for a user-specified number of iterations, which we set to 5 here. 
# Further details on about this algorithm, can be found it the package's [documentation][1].
from gpytoolbox import remesh_botsch
v_new, f_new = remesh_botsch(v_np, f_np, i=5, h=target_l, project=True)

# put v_new and f_new on gpu again
# Create torch tensors from the numpy arrays
v_new_t = torch.from_numpy(v_new).float().cuda()
f_new_t = torch.from_numpy(f_new).int().cuda()

# # Move tensors to the GPU
v_new_t = v_new_t.to('cuda')
f_new_t = f_new_t.to('cuda')

# The new vertices and faces must now be passed to our Mitsuba `Mesh`. If the mesh has other attributes (e.g. UV coordinates), they also need to be updated. By default, Mitsuba will reset these to 0 if the vertex or face count is altered.
params = mi.traverse(scene_source)
# params['shape.vertex_positions'] = mi.Float(v_new.flatten().astype(np.float32))
# params['shape.faces'] = mi.Int(f_new.flatten())
params['shape.vertex_positions'] = dr.ravel(mi.TensorXf(v_new_t.to(torch.float32)))
params['shape.faces'] = dr.ravel(mi.TensorXf(f_new_t.to(torch.float32)))
params.update();

# mesh topology is different, we need new latent variable
ls = mi.ad.LargeSteps(params['shape.vertex_positions'], params['shape.faces'], lambda_)
opt = mi.ad.Adam(lr=1e-1, uniform=True)
opt['u'] = ls.to_differential(params['shape.vertex_positions'])

torch.cuda.empty_cache()

# optimisation
# iterations = 150 if 'PYTEST_CURRENT_TEST' not in os.environ else 5
# for it in range(iterations):
#     loss = mi.Float(0.0)

#     # Retrieve the vertex positions from the latent variable
#     params['shape.vertex_positions'] = ls.from_differential(opt['u'])
#     params.update()

#     current_img = mi.render(scene_source, params, seed=it, spp=16)

#     # L1 Loss
#     loss = dr.mean(dr.abs(current_img - mi.TensorXf(input_img.squeeze(0))))
#     torch.cuda.empty_cache()

#     dr.backward(loss)
#     opt.step()

#     print(f"Iteration {1+it:03d}: Loss = {loss[0]:6f}", end='/r')
    
params['shape.vertex_positions'] = ls.from_differential(opt['u'])
params.update();

## Final result

Finally, let's compare our end result (bottom) to our reference views (top).

In [ ]:
final_img = mi.render(scene_source, spp=128)

fig, ax = plt.subplots(nrows=2, figsize=(5, 10))
ax[0].set_title("Reference", y=0.3, x=-0.005, rotation=90, fontsize=20)
ax[1].set_title("Optimized shape", y=0.2, x=-0.005, rotation=90, fontsize=20)
for i, img in enumerate((ref_img, final_img)):
    ax[i].imshow(mi.util.convert_to_bitmap(img))
    ax[i].axis('off')

Note that the results could be further improved by e.g. using more input views, or using a less agressive step size and more iterations.

## See also

- [<code>mitsuba.ad.LargeSteps</code>][1]
- [<code>direct_reparam</code> plugin][2]
- [<code>batch</code> plugin][3]

[1]: https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.LargeSteps
[2]: https://mitsuba.readthedocs.io/en/latest/src/generated/plugins_integrators.html#reparameterized-direct-integrator-direct-reparam
[3]: https://mitsuba.readthedocs.io/en/latest/src/generated/plugins_sensors.html#batch-sensor-batch